# SDK Usage Notes 

For CLI notes and other information, see the README.md file in the root of the repository.

1. This notebook shows a minimal end-to-end example: from logging in through placing and canceling a spot bid.  
2. You can adapt or expand these steps. For instance, you might script your entire workflow using FlowTaskManager to parse configuration and automatically place bids.  

You are now ready to leverage the Flow system for programmatic tasks, resource provisioning, and persistent storage management. Happy building!

# Installation and Quickstart

## Installation
1. Ensure you have Python 3.11 or higher installed.  
2. Install dependencies either by:  
   - Using pdm (preferred):  
     » pdm install  
   - Or manually with pip:  
     » pip install -r requirements.txt  

## Quick-Start
1. Configure Foundry environment variables, as described in src/flow/config/flow_config.py:  

  **Authentication (choose one option):**

   - Option 1 (Recommended): API Key
      - export FOUNDRY_API_KEY='your_api_key' # Found in FCP Console → Settings → API Keys tab

   - Option 2: Email/Password
      - export FOUNDRY_EMAIL='your_email@example.com'  
      - export FOUNDRY_PASSWORD='your_password'  


2. Submit an example task using flow_example.yaml:
   » flow submit flow_example.yaml

3. Check the status of your submitted task:
   » flow status

---

# Flow Quickstart Notebook

This notebook demonstrates how to interact with the Flow system to:
1. Authenticate and instantiate a Foundry client.
2. Retrieve user & project information.
3. Retrieve auctions and place/cancel spot bids.
4. (Optionally) create persistent storage using StorageManager.

Make sure you have your Flow library installed.

In [1]:
# If not installed, uncomment and install the Flow package.
# !pdm install
# !pip install flow

In [2]:
# 1) Imports and Initialization

import logging
import os
from typing import Any, Dict, List

# 3rd-party library for printing tables. Install via:
# !pip install tabulate
from tabulate import tabulate

# Example modules from your Flow environment.
from flow.clients.foundry_client import FoundryClient
from flow.managers.storage_manager import StorageManager
from flow.models import PersistentStorage, PersistentStorageCreate
from flow.models import SshKey

# Optional: Additional managers you can look into or extend if you need advanced functionality
# from flow.managers.task_manager import FlowTaskManager
# from flow.managers.auction_finder import AuctionFinder
# from flow.managers.bid_manager import BidManager


# Configure logging to show detailed information (INFO level).
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s | %(name)s | %(levelname)s | %(message)s"
)

logger = logging.getLogger(__name__)
logger.info("Starting Flow SDK Quickstart...")

2025-02-04 14:02:16,383 | __main__ | INFO | Starting Flow SDK Quickstart...


In [3]:
def check_required_env_vars():
    """
    Checks that either an API key or email/password environment variables are set.
    Raises an EnvironmentError with a clear message if neither is set.
    """
    import os

    api_key = os.getenv("FOUNDRY_API_KEY")
    email = os.getenv("FOUNDRY_EMAIL")
    password = os.getenv("FOUNDRY_PASSWORD")

    if not api_key and (not email or not password):
        raise EnvironmentError(
            "Missing credentials. Please set either:\n"
            " • FOUNDRY_API_KEY, or\n"
            " • FOUNDRY_EMAIL and FOUNDRY_PASSWORD."
        )
    else:
        print("Credentials present (API key or email/password).")


# Just to confirm these are configured before anything else:
check_required_env_vars()

Credentials present (API key or email/password).


In [4]:
# 1) Initialize Foundry Client
def initialize_foundry_client() -> FoundryClient:
    """Initializes and returns a FoundryClient instance.

    Returns:
        A FoundryClient configured with credentials from environment variables.
        Uses API key if available; otherwise, uses email/password.
    """
    import os
    from flow.clients.foundry_client import FoundryClient

    email = os.getenv("FOUNDRY_EMAIL")
    password = os.getenv("FOUNDRY_PASSWORD")
    api_key = os.getenv("FOUNDRY_API_KEY")

    # Either api_key is provided, or email/password must be valid.
    if not api_key and (not email or not password):
        logger.error(
            "Either FOUNDRY_API_KEY or FOUNDRY_EMAIL/FOUNDRY_PASSWORD must be set."
        )
        raise ValueError("Missing Foundry credentials in environment variables.")

    client = FoundryClient(email=email, password=password, api_key=api_key)
    logger.info("Foundry client initialized successfully.")
    return client

In [5]:
foundry_client = initialize_foundry_client()

2025-02-04 14:02:18,289 | flow.clients.authenticator | INFO | User authenticated successfully; access token retrieved.
2025-02-04 14:02:20,552 | flow.clients.fcp_client | INFO | FCPClient initialized successfully for user_id=7f59f654-8f1a-47c3-9b2f-85e89b30575e
2025-02-04 14:02:20,553 | flow.clients.foundry_client | INFO | FoundryClient initialized successfully.
2025-02-04 14:02:20,554 | __main__ | INFO | Foundry client initialized successfully.


In [6]:
# 2) Retrieve User and Project Information


def get_user_and_project(
    foundry_client: FoundryClient,
    project_name: str,
) -> Dict[str, Any]:
    """Fetches user details and the first available project.

    Args:
        foundry_client: An authenticated FoundryClient instance.

    Returns:
        A dictionary containing user info and selected project info.

    Raises:
        RuntimeError: If no projects are found for the user.
    """
    user = foundry_client.get_user()
    logger.info("Authenticated as User ID: %s", user.id)

    project = foundry_client.get_project_by_name(project_name=project_name)
    if not project:
        logger.error("Project not found for this user.")
        raise RuntimeError("Specified project not found for this user.")

    logger.info("Selected Project: %s (ID: %s)", project.name, project.id)

    return {"user": user, "selected_project": project}

In [8]:
foundry_project_name = input("Please enter your Foundry project name: ")

Please enter your Foundry project name:  test


In [9]:
# Ensure that the user has set the foundry_project_name variable.
try:
    foundry_project_name
except NameError:
    raise NameError(
        "The variable 'foundry_project_name' is not defined. "
        "Please define it with your Foundry project name before proceeding. "
        "For example:\n\nfoundry_project_name = 'your_project_name_here' or via the interactive prompt above"
    )

In [10]:
info = get_user_and_project(
    foundry_client=foundry_client,
    project_name=foundry_project_name,
)
user = info["user"]
selected_project = info["selected_project"]
project_id = selected_project.id

2025-02-04 14:14:29,078 | __main__ | INFO | Authenticated as User ID: 7f59f654-8f1a-47c3-9b2f-85e89b30575e
2025-02-04 14:14:29,975 | flow.clients.fcp_client | ERROR | No project found with name='<your_foundry_project_name>'


ValueError: No project found with name: <your_foundry_project_name>

In [ ]:
# # 3) List Instances in the Selected Project

# We fetch all instances grouped by category, then print the total count and
# display them in a structured table for clarity.


def list_instances_in_project(
    foundry_client: FoundryClient, project_id: str
) -> Dict[str, List[Any]]:
    """Fetches instances for a given project, returning them as a category->list dictionary.

    Args:
        foundry_client: An authenticated FoundryClient instance.
        project_id: The unique identifier of the project.

    Returns:
        A dictionary where keys are categories and values are lists of instance objects.
    """
    instances_dict = foundry_client.get_instances(project_id=project_id)
    logger.info("Successfully retrieved instances for project ID: %s", project_id)
    return instances_dict


def print_total_instance_count(instances_dict: Dict[str, List[Any]]) -> None:
    """Calculates and logs the total number of instances across all categories.

    Args:
        instances_dict: A dictionary containing category->list of instance objects.
    """
    total_instances = sum(len(instances) for instances in instances_dict.values())
    logger.info("Total Instances in project: %d", total_instances)


def print_instances_table(instances_dict: Dict[str, List[Any]]) -> None:
    """Prints all instances in a structured table format.

    Args:
        instances_dict: A dictionary containing category->list of instance objects.
                        Each instance is expected to have attributes:
                        instance_id, name, and instance_status.
    """
    table_rows = []
    for category, instance_list in instances_dict.items():
        for instance in instance_list:
            table_rows.append(
                [
                    category,
                    instance.instance_id,
                    instance.name,
                    instance.instance_status,
                ]
            )

    headers = ["Category", "Instance ID", "Name", "Status"]
    table_str = tabulate(table_rows, headers=headers, tablefmt="github")
    print(table_str)  # Or use logging if you prefer to log this output.

In [ ]:
# 3) List Instances in the Selected Project
instances_dict = list_instances_in_project(foundry_client, project_id)
print_total_instance_count(instances_dict)
print_instances_table(instances_dict)

logger.info("Flow environment set up and instance listing completed successfully.")

In [ ]:
# Optional: Example storage manager usage
storage_manager = StorageManager(foundry_client=foundry_client)
logger.info("Storage manager initialized.")

In [ ]:
# 4) Retrieve Spot Auctions
from tabulate import tabulate
from typing import List
from flow.models import Auction


def display_auctions_pretty(auctions: List[Auction], max_rows: int = 5) -> None:
    """
    Display the given auctions in a table.

    Args:
        auctions: A list of Auction objects.
        max_rows: Maximum number of auctions to display to avoid overwhelming output.
                  Set to a large number (or None) to show all.
    """
    if not auctions:
        print("No auctions to display.")
        return

    # Limit how many rows we show (for readability).
    auctions_to_show = auctions[:max_rows]

    # Prepare rows for tabulation.
    table_data = []
    for idx, auction in enumerate(auctions_to_show, start=1):
        table_data.append(
            [
                idx,
                auction.cluster_id,
                auction.gpu_type or "N/A",
                auction.inventory_quantity or 0,
                auction.last_price if auction.last_price is not None else "N/A",
                auction.region or "N/A",
            ]
        )

    # Define table headers.
    headers = ["#", "Cluster ID", "GPU Type", "Inventory Qty", "Last Price", "Region"]

    # Print in a compact, readable table.
    print(tabulate(table_data, headers=headers, tablefmt="fancy_grid"))

    # Inform the user if there are more rows not shown.
    if len(auctions) > max_rows:
        print(f"\nShowing only the first {max_rows} of {len(auctions)} auctions.")
        print("Increase `max_rows` if you need to see more.\n")

In [ ]:
auctions = foundry_client.get_auctions(project_id=project_id)
display_auctions_pretty(auctions, max_rows=10)

In [ ]:
# 5) Retrieve SSH Keys


def display_ssh_keys(ssh_keys: List[SshKey]) -> None:
    """Display available SSH keys in a user-friendly table.

    Args:
        ssh_keys: A list of SshKey objects.
    """
    if not ssh_keys:
        logger.info("No SSH keys to display.")
        return

    table_data = []
    for idx, key in enumerate(ssh_keys, start=1):
        table_data.append([idx, key.id, key.name])

    headers = ["#", "Key ID", "Name"]
    print(tabulate(table_data, headers=headers, tablefmt="fancy_grid"))
    logger.info("Displayed %d SSH key(s).", len(ssh_keys))


def get_first_ssh_key(foundry_client: FoundryClient, project_id: str) -> SshKey:
    """Retrieves the first available SSH key for demonstration.

    Args:
        foundry_client: The client used to interface with Foundry.
        project_id: The project ID to fetch SSH keys from.

    Returns:
        The first SSH key found.

    Raises:
        RuntimeError: If no SSH keys exist for the given project.
    """
    ssh_keys = foundry_client.get_ssh_keys(project_id=project_id)
    if not ssh_keys:
        logger.error("No SSH keys found. Upload or create one in the Foundry UI.")
        raise RuntimeError("No SSH keys found.")

    display_ssh_keys(ssh_keys)
    chosen_key = ssh_keys[0]
    logger.info("Using SSH key '%s' (ID: %s)", chosen_key.name, chosen_key.id)
    return chosen_key


ssh_key = get_first_ssh_key(foundry_client, project_id)

In [ ]:
# 6) Place a Spot Bid

# For demonstration, we place a bid on the first available auction (if any).
if not auctions:
    raise RuntimeError("No auctions available to bid on.")
auction = auctions[0]

In [ ]:
auction

In [ ]:
import random
import string


def generate_random_suffix(length=8):
    """Generates a random alphanumeric suffix of a specified length."""
    letters_and_digits = string.ascii_lowercase + string.digits
    return "".join(random.choice(letters_and_digits) for i in range(length))


random_suffix = generate_random_suffix()
order_name = f"demo-spot-order-0-{random_suffix}"
print(order_name)

In [ ]:
limit_price_cents = 999  # e.g. $9.99
instance_quantity = 1
startup_script = ""  # e.g. could pass shell commands to configure the instance

# Construct a bid payload
from flow.models import BidPayload

bid_payload = BidPayload(
    cluster_id=auction.id,
    instance_quantity=instance_quantity,
    instance_type_id=auction.instance_type_id,
    limit_price_cents=limit_price_cents,
    order_name=order_name,
    project_id=project_id,
    ssh_key_ids=[ssh_key.id],
    startup_script=startup_script,
    user_id=user.id,
)

# Now place the bid using the FCPClient under the hood
bid_response = foundry_client.place_bid(project_id=project_id, bid_payload=bid_payload)

print("Bid placed successfully:", bid_response.model_dump())

In [ ]:
# 7) Cancel a Spot Bid (optional)

# If you want to cancel what you just placed:
bid_id = bid_response.id
print(f"Cancelling bid with ID: {bid_id}")

foundry_client.cancel_bid(project_id=project_id, bid_id=bid_id)
print("Bid cancelled successfully!")

In [ ]:
# 8) Create Persistent Storage (optional)

# If your Flow config or use case includes creating persistent storage, you can do so
# using the StorageManager. For example:

persistent_storage = PersistentStorage(
    create=PersistentStorageCreate(
        volume_name="test-volume", size=10, size_unit="GB"  # e.g. 10GB
    )
)

# Attempt to create the disk
disk_attachment = storage_manager.handle_persistent_storage(
    project_id=project_id,
    persistent_storage=persistent_storage,
    region_id=None,  # or specify a region if you know it
)

if disk_attachment:
    print("Persistent storage created with disk ID:", disk_attachment.disk_id)
else:
    print("No persistent storage was requested/created.")

In [ ]:
# 9) Retrieve Current Bids (to check status or cleanup)

from flow.models import Bid


def display_bids(bids: List[Bid]) -> None:
    """Displays the current bids in the project in a tabular format.

    Args:
        bids: A list of Bid objects.
    """
    if not bids:
        logger.info("No bids found in the project.")
        print("No bids found in the project.")
        return

    table_data = []
    for idx, b in enumerate(bids, start=1):
        table_data.append([idx, b.name, b.id, b.status])

    headers = ["#", "Bid Name", "Bid ID", "Status"]
    print(tabulate(table_data, headers=headers, tablefmt="fancy_grid"))
    logger.info("Displayed %d bid(s).", len(bids))


def retrieve_and_display_bids(foundry_client: FoundryClient, project_id: str) -> None:
    """Retrieves and displays all bids for the project.

    Args:
        foundry_client: The client used to interface with Foundry.
        project_id: The project ID.
    """
    bids = foundry_client.get_bids(project_id=project_id)
    logger.info("Found %d bids in the project.", len(bids))
    display_bids(bids)


retrieve_and_display_bids(foundry_client, project_id)